# Showing why PCA preprocessing in single-cell genomics is a terrible idea

For the manuscript. For now only have 14 datasets, but the example sufficies.

In [1]:
import glob
from pcaeval.pca_eval import evaluate_anndata_file_list, plot_sing_vals_exp_var

In [2]:
wd = '/home/davi/Bioinfo/PCA_distortion/'
data_dir = '/home/davi/Bioinfo/BenchmarkDatasets/data/'
save_dir = '/home/davi/Bioinfo/BenchmarkDatasets/pca_distortion_results/'

adata_files = glob.glob(data_dir + '*.h5ad')
adata_files

['/home/davi/Bioinfo/BenchmarkDatasets/data/C. elegans.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Embryoid Body.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Hydra sp.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Mouse brain.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Bone marrow (M).h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Glioblastoma.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/PBMC (Lupus).h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/PBMCs (10k).h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Zebrafish.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Stimulated T cells.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/PBMCs (68k).h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Paul 2015.h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Bone marrow (H).h5ad',
 '/home/davi/Bioinfo/BenchmarkDatasets/data/Mouse Organogenesis.h5ad']

In [4]:
results_dict = evaluate_anndata_file_list(adata_files, n_pcs=100,
 metric='euclidean', n_neighbors=15, save_intermediate=False, save_dir=save_dir, verbose=True,
  not_normalize_idx=[13], return_dict=True)

: 

: 

In [ ]:
plot_sing_vals_exp_var(results_dict, fontsize=10)

In [ ]:
for i, dataset_name in enumerate(results_dict):
    print('\n \n  --- ' + str(dataset_name) + ' --- ' +
 '\n Spearman R correlation between the k-nearest-neighbors graphs learned from the data and the Principal Components: %f'%results_dict['graph_correlation'] +
 '\n Spearman R correlation between the geodesic distances in UMAP embeddings learned from these graphs %f'%results_dict['embedding_correlation'] +
 '\n Total explained variance with the first 100 PCs: %f'%results_dict['explained_variance'].max() +
 '\n Adjusted rand index (ARI) between clustering on top 100 PCs and on full data: %f'%results_dict['adjusted_rand_score'])